## HyperOpt Remote Training with Child Runs

https://github.com/hyperopt/hyperopt-sklearn/blob/master/notebooks/Demo-Iris.ipynb

In [12]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.33.0 to work with mm-aml-dev


In [13]:
import os, shutil
folder_name = 'train'
script_folder = os.path.join(os.getcwd(), folder_name)
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/chr-demo2/train


In [14]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, 'hyperOptWithChildRuns')

In [15]:
script_file = os.path.join(script_folder,"train.py")
print(script_file)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/chr-demo2/train/train.py


In [97]:
%%writefile $script_file

from azureml.core import Experiment, Run, Model, Dataset
import os
os.environ['OMP_NUM_THREADS'] = '1'

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import hpsklearn
import hpsklearn.demo_support
import hyperopt.tpe
import pandas as pd
import numpy as np
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import joblib


directory = 'outputs'
if not os.path.exists(directory):
    os.mkdir(directory)
        
def scatter_error_vs_time(estimator, ax):
    losses = estimator.trials.losses()
    ax.set_ylabel('Validation error rate')
    ax.set_xlabel('Iteration')
    ax.scatter(list(range(len(losses))), losses)


def plot_minvalid_vs_time(estimator, ax, ylim=None):
    losses = estimator.trials.losses()
    ts = list(range(1, len(losses)))
    mins = [np.min(losses[:ii]) for ii in ts]
    ax.set_ylabel('min(Validation error rate to-date)')
    ax.set_xlabel('Iteration')
    if ylim:
        ax.set_ylim(*ylim)
    ax.plot(ts, mins)


class MyplotHelper(object):

    def __init__(self, estimator, mintodate_ylim):
        self.estimator = estimator
        self.fig, self.axs = plt.subplots(1, 2)
        self.post_iter_wait = .5
        self.mintodate_ylim = mintodate_ylim

    def post_iter(self):
        self.axs[0].clear()
        self.axs[1].clear()
        scatter_error_vs_time(self.estimator, self.axs[0])
        plot_minvalid_vs_time(self.estimator, self.axs[1],
                              ylim=self.mintodate_ylim)
        display.clear_output()
        print('saving image')
        self.fig.savefig('./outputs/output.png')
        #display.display(self.fig)
        time.sleep(self.post_iter_wait)

    def post_loop(self):
        self.fig.savefig('./outputs/outputfinal.png')
        display.clear_output()

run = Run.get_context() 

print('creating estimator')
estimator = hpsklearn.HyperoptEstimator(
    preprocessing=hpsklearn.components.any_preprocessing('pp'),
    classifier=hpsklearn.components.any_classifier('clf'),
    algo=hyperopt.tpe.suggest,
    trial_timeout=15.0, # seconds
    max_evals=15,
    )

print('loading dataset')
# PREPARE TRAINING AND TEST DATA
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['species_name'] = pd.Categorical.from_codes(iris.target, iris.target_names)
y = df_iris['species_name']
X = df_iris.drop(['species_name'], axis=1)

# TRAIN AND TEST DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Demo version of estimator.fit()
fit_iterator = estimator.fit_iter(X_train,y_train)
fit_iterator.__next__()
plot_helper = MyplotHelper(estimator,
                                                mintodate_ylim=(-.01, .10))
while len(estimator.trials.trials) < estimator.max_evals:
    child = run.child_run()
    child.log(name= 'max evals', value = estimator.max_evals)
    child.log(name ='run', value = estimator.max_evals)
    fit_iterator.send(1) # -- try one more model
    
    losses = estimator.trials.losses()
    index = len(losses)
    
    child.log(name= 'index', value = index)
    
    plot_helper.post_iter()

    child.upload_file(name = './outputs/output.png', path_or_stream = './outputs/output.png')
    child.complete()
    
plot_helper.post_loop()

# -- Model selection was done on a subset of the training data.
# -- Now that we've picked a model, train on all training data.
estimator.retrain_best_model_on_full_data(X_train, y_train)

print(estimator._best_preprocs)
print('***************************')
print('Best preprocessing pipeline:')
for pp in estimator._best_preprocs:
    print(pp)
    run.log(name = 'best prepossing', value = pp)
print('\n')
print('Best classifier:\n', estimator._best_learner)

run.log(name = 'best classifer', value = estimator._best_learner)
test_predictions = estimator.predict(X_test)
acc_in_percent = 100 * np.mean(test_predictions == y_test)
print('\n')
print('Prediction accuracy in generalization is %.1f%%' % acc_in_percent)
run.log(name = 'accuracy', value = acc_in_percent)

#run.complete()

print('*****************************')

# Save the trained model
model_file = 'model.pkl'
joblib.dump(value=estimator, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()


# Register the model
run.register_model(model_path='outputs/model.pkl', model_name='iris-hypteropt')

print('Model trained and registered.')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/chr-demo2/train/train.py


In [98]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException

compute_name =  "mm-cluster"
print(compute_name)

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D13",
                                                   min_nodes=0, 
                                                   max_nodes=1)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=120)

mm-cluster


In [99]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
hyperOpt_env = Environment("hyperOpt-experiment-env")
hyperOpt_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
hyperOpt_env.docker.enabled = True # Use a docker container


# Create a set of package dependencies (conda or pip as required)
hyperOpt_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip', 'IPython'],
                                              pip_packages=['azureml-sdk','numpy', 'joblib', 'sklearn', 'git+https://github.com/hyperopt/hyperopt-sklearn'])

# Add the dependencies to the environment
hyperOpt_env.python.conda_dependencies = hyperOpt_packages

print(hyperOpt_env.name, 'defined.')

# Register the environment
hyperOpt_env.register(workspace=ws)
print('registered')

hyperOpt-experiment-env defined.
registered


In [100]:
# Create a script config
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

registered_env = Environment.get(ws, 'hyperOpt-experiment-env')

script_config = ScriptRunConfig(source_directory=script_folder,
                                script='train.py',
                                environment=registered_env,
                                compute_target=compute_name) 

# submit the experiment
run = experiment.submit(config=script_config)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…